# ETL & Datawarehouse
**Purpose** : load, process and save data from a datalake (S3) to a datawarehouse (RDS)

In [1]:
# import 
import os
import sqlalchemy
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [ ]:
# load dotenv variables


In [ ]:
# engine : connected to our db